In [ ]:
# In[1]:
import numpy as np
import xarray as xr
import pandas as pd
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
# %%
# define function
import src.SAT_function as data_process
import src.Data_Preprocess as preprocess

In [ ]:
import src.slurm_cluster as scluster
client, scluster = scluster.init_dask_slurm_cluster()

In [ ]:
def func_mk(x):
    """
    Mann-Kendall test for trend
    """
    results = data_process.mk_test(x)
    slope = results[0]
    p_val = results[1]
    return slope, p_val
 

In [ ]:
dir1 ='/work/mh0033/m301036/Land_surf_temp/Disentangling_OBS_SAT_trend/Figure4_Validation/MPI_ESM/'
MPI_ESM_forced = xr.open_dataset(dir1 + 'GSAT_MPI_ESM_Forced_anomalies_1850_2022.nc')
MPI_ESM_internal = xr.open_dataset(dir1 + 'GSAT_MPI_ESM_Internal_Variability_anomalies_1850_2022.nc')

In [ ]:
MPI_ESM_internal

### Calculate the trend in 10, 30, 60yr

In [ ]:
variable_name = ['10yr', '30yr', '60yr']
# define the function
def separate_data_into_intervals(data, start_year, end_year, time_interval):
    """
    This function is used to separate the data into different time intervals
    """
    # create a dictionary to store the data
    data_dict = {}
    for i in range(len(variable_name)):
        # calculate the start year and end year
        start_year = time_interval[variable_name[i]][0]
        end_year = time_interval[variable_name[i]][1]
        # select the data
        data_dict[variable_name[i]] = data.sel(year=slice(str(start_year), str(end_year)))
    return data_dict

In [ ]:
time_interval = {
    "10yr":(2013,2022),
    "30yr":(1993,2022),
    "60yr":(1963,2022)
}
start_year = 1950
end_year   = 2022

In [ ]:
MPI_ESM_forced_data_dict = separate_data_into_intervals(MPI_ESM_forced, start_year, end_year, time_interval)
MPI_ESM_unforced_data_dict = separate_data_into_intervals(MPI_ESM_internal, start_year, end_year, time_interval)

In [ ]:
# Calculate the trend and p-value for each time interval of each realization
trend_dict = {}
pvalue_dict = {}

for i in range(len(variable_name)):
    data_var = MPI_ESM_forced_data_dict[variable_name[i]]['tas']
    
    slope, p_values = xr.apply_ufunc(
        func_mk,
        data_var,
        input_core_dims=[["year"]],
        output_core_dims=[[], []],
        vectorize=True,
        dask="parallelized",
        output_dtypes=[float, float],
        dask_gufunc_kwargs={'allow_rechunk': True}
    )
    trend_dict[variable_name[i]] = slope
    pvalue_dict[variable_name[i]] = p_values

In [ ]:
trend_dict

In [ ]:
trend_annual_np = {}
pvalue_annual_np = {}

for i in range(len(variable_name)):
    trend_annual_np[variable_name[i]] = trend_dict[variable_name[i]].values
    pvalue_annual_np[variable_name[i]] = pvalue_dict[variable_name[i]].values
    
trend_annual_np['10yr']

In [ ]:
trend_annual_da = {}
pvalue_annual_da = {}

for interval, data in trend_annual_np.items():
    trend_annual_da[interval] = xr.DataArray(data, dims=["run","lat", "lon"], coords={"run": MPI_ESM_forced_data_dict[interval].run, 
    "lat": MPI_ESM_forced_data_dict[interval].lat, "lon": MPI_ESM_forced_data_dict[interval].lon})
for interval, data in pvalue_annual_np.items():
    pvalue_annual_da[interval] = xr.DataArray(data, dims=["run","lat", "lon"], coords={"run": MPI_ESM_forced_data_dict[interval].run, 
    "lat": MPI_ESM_forced_data_dict[interval].lat, "lon": MPI_ESM_forced_data_dict[interval].lon})

In [ ]:
# out put the forced trend and pvalue
dir_out ='/work/mh0033/m301036/Land_surf_temp/Disentangling_OBS_SAT_trend/Figure2/MPI_ESM/trend/'

for interval, data in trend_annual_da.items():
    data.to_netcdf(dir_out + 'MPI_ESM_forced_' + interval + '_trend.nc')
for interval, data in pvalue_annual_da.items():
    data.to_netcdf(dir_out + 'MPI_ESM_forced_' + interval + '_pvalue.nc')

In [ ]:
trend_annual_da

In [ ]:
# Put the original data into a dictionary
pvalue_annual_da

### unforced trend calculation

In [ ]:
unforced_trend_dict = {}
unforced_pvalue_dict = {}

for i in range(len(variable_name)):
    data_var = MPI_ESM_unforced_data_dict[variable_name[i]]['tas']
    
    slope, p_values = xr.apply_ufunc(
        func_mk,
        data_var,
        input_core_dims=[["year"]],
        output_core_dims=[[], []],
        vectorize=True,
        dask="parallelized",
        output_dtypes=[float, float],
        dask_gufunc_kwargs={'allow_rechunk': True}
    )
    unforced_trend_dict[variable_name[i]] = slope
    unforced_pvalue_dict[variable_name[i]] = p_values

In [ ]:
unforced_trend_np = {}
unforced_pvalue_np = {}

for i in range(len(variable_name)):
    unforced_trend_np[variable_name[i]] = unforced_trend_dict[variable_name[i]].values
    unforced_pvalue_np[variable_name[i]] = unforced_pvalue_dict[variable_name[i]].values

unforced_trend_da = {}
unforced_pvalue_da = {}

for interval, data in unforced_trend_np.items():
    unforced_trend_da[interval] = xr.DataArray(data, dims=["run","lat", "lon"], coords={"run": MPI_ESM_unforced_data_dict[interval].run, 
    "lat": MPI_ESM_unforced_data_dict[interval].lat, "lon": MPI_ESM_unforced_data_dict[interval].lon})
for interval, data in unforced_pvalue_np.items():
    unforced_pvalue_da[interval] = xr.DataArray(data, dims=["run","lat", "lon"], coords={"run": MPI_ESM_unforced_data_dict[interval].run, 
    "lat": MPI_ESM_unforced_data_dict[interval].lat, "lon": MPI_ESM_unforced_data_dict[interval].lon})


In [ ]:
unforced_trend_da

In [ ]:
dir_out ='/work/mh0033/m301036/Land_surf_temp/Disentangling_OBS_SAT_trend/Figure4_Validation/MPI_ESM/trend/'
for interval, data in unforced_trend_da.items():
    data.to_netcdf(dir_out + 'MPI_ESM_unforced_' + interval + '_trend.nc')
for interval, data in unforced_pvalue_da.items():
    data.to_netcdf(dir_out + 'MPI_ESM_unforced_' + interval + '_pvalue.nc')

In [ ]:
dir_input = '/work/mh0033/m301036/Land_surf_temp/Disentangling_OBS_SAT_trend/Supp_Figure1_Forced/data/MPI_ESM_Single/'

MPI_ESM_10yr_trend   = xr.open_dataset(dir_input + 'MPI_ESM_annual_10yr_ensemble_mean_trend.nc')
MPI_ESM_30yr_trend   = xr.open_dataset(dir_input + 'MPI_ESM_annual_30yr_ensemble_mean_trend.nc')
MPI_ESM_60yr_trend   = xr.open_dataset(dir_input + 'MPI_ESM_annual_60yr_ensemble_mean_trend.nc')

In [ ]:
# pattern correlation betwenn observed forced pattern vs. Model simulated forced pattern
import scipy.stats as stats
# 10yr forced trend in each realization .vs. MPI_ESM ensemble trend MPI_ESM_10yr_trend
MPI_ESM_forced_10yr_trend = trend_annual_da['10yr']
MPI_ESM_forced_30yr_trend = trend_annual_da['30yr']
MPI_ESM_forced_60yr_trend = trend_annual_da['60yr']

MPI_ESM_unforced_10yr_trend = unforced_trend_da['10yr']
MPI_ESM_unforced_30yr_trend = unforced_trend_da['30yr']
MPI_ESM_unforced_60yr_trend = unforced_trend_da['60yr']

# Pattern correlations calculation
trend_pattern_correlation_10yr = []
for i in range(len(trend_annual_da['10yr'].run)):
    trend_pattern_correlation_10yr.append(stats.pearsonr(MPI_ESM_forced_10yr_trend.isel(run=i).values.flatten(), MPI_ESM_10yr_trend['tas'].values.flatten())[0])

trend_pattern_correlation_30yr = []
for i in range(len(trend_annual_da['30yr'].run)):
    trend_pattern_correlation_30yr.append(stats.pearsonr(MPI_ESM_forced_30yr_trend.isel(run=i).values.flatten(), MPI_ESM_30yr_trend['tas'].values.flatten())[0])

trend_pattern_correlation_60yr = []
for i in range(len(trend_annual_da['60yr'].run)):
    trend_pattern_correlation_60yr.append(stats.pearsonr(MPI_ESM_forced_60yr_trend.isel(run=i).values.flatten(), MPI_ESM_60yr_trend['tas'].values.flatten())[0])
    

In [ ]:
trend_pattern_correlation_10yr

In [ ]:
with open('pattern_correlations_vs_mean_of_trend.txt', 'w') as file:
    file.write('10-year Trend Pattern Correlations:\n')
    for correlation in trend_pattern_correlation_10yr:
        file.write(f"{correlation}\n")

    file.write('\n30-year Trend Pattern Correlations:\n')
    for correlation in trend_pattern_correlation_30yr:
        file.write(f"{correlation}\n")

    file.write('\n60-year Trend Pattern Correlations:\n')
    for correlation in trend_pattern_correlation_60yr:
        file.write(f"{correlation}\n")

In [ ]:
# save the pattern correlation into text file
# Save to a text file
with open('pattern_correlations.txt', 'w') as file:
    file.write('10-year Trend Pattern Correlations:\n')
    for correlation in trend_pattern_correlation_10yr:
        file.write(f"{correlation}\n")

    file.write('\n30-year Trend Pattern Correlations:\n')
    for correlation in trend_pattern_correlation_30yr:
        file.write(f"{correlation}\n")

    file.write('\n60-year Trend Pattern Correlations:\n')
    for correlation in trend_pattern_correlation_60yr:
        file.write(f"{correlation}\n")

In [ ]:
# Input the observational forced and unforced trend wrt. MPI_ESM-CM6A-LR
dir_obs_MPI_ESM = '/work/mh0033/m301036/Land_surf_temp/Disentangling_OBS_SAT_trend/Figure1/data/'

Obs_MPI_ESM_forced_10yr_trend = xr.open_dataset(dir_obs_MPI_ESM + 'HadCRUT5_annual_forced_10yr_trend_MPI_ESM.nc')
Obs_MPI_ESM_forced_30yr_trend = xr.open_dataset(dir_obs_MPI_ESM + 'HadCRUT5_annual_forced_30yr_trend_MPI_ESM.nc')
Obs_MPI_ESM_forced_60yr_trend = xr.open_dataset(dir_obs_MPI_ESM + 'HadCRUT5_annual_forced_60yr_trend_MPI_ESM.nc')

Obs_MPI_ESM_unforced_10yr_trend = xr.open_dataset(dir_obs_MPI_ESM + 'HadCRUT5_annual_internal_10yr_trend_MPI_ESM.nc')
Obs_MPI_ESM_unforced_30yr_trend = xr.open_dataset(dir_obs_MPI_ESM + 'HadCRUT5_annual_internal_30yr_trend_MPI_ESM.nc')
Obs_MPI_ESM_unforced_60yr_trend = xr.open_dataset(dir_obs_MPI_ESM + 'HadCRUT5_annual_internal_60yr_trend_MPI_ESM.nc')

In [ ]:
Obs_MPI_ESM_unforced_10yr_trend

In [ ]:
# calculate the pattern correlation between observed and model's each realization simulated forced and unforced trend
OBS_trend_pattern_correlation_10yr = []
for i in range(len(trend_annual_da['10yr'].run)):
    print(i)
    OBS_trend_pattern_correlation_10yr.append(stats.pearsonr(MPI_ESM_forced_10yr_trend.isel(run=i).values.flatten(), Obs_MPI_ESM_forced_10yr_trend['tas'].values.flatten())[0])

OBS_trend_pattern_correlation_30yr = []
for i in range(len(trend_annual_da['30yr'].run)):
    print(i)
    OBS_trend_pattern_correlation_30yr.append(stats.pearsonr(MPI_ESM_forced_30yr_trend.isel(run=i).values.flatten(), Obs_MPI_ESM_forced_30yr_trend['tas'].values.flatten())[0])

OBS_trend_pattern_correlation_60yr = []
for i in range(len(trend_annual_da['60yr'].run)):
    print(i)
    OBS_trend_pattern_correlation_60yr.append(stats.pearsonr(MPI_ESM_forced_60yr_trend.isel(run=i).values.flatten(), Obs_MPI_ESM_forced_60yr_trend['tas'].values.flatten())[0])

In [ ]:
# Save to a text file
with open('obsForced_MPI_ESMforced_pattern_correlations.txt', 'w') as file:
    file.write('10-year Trend Pattern Correlations:\n')
    for correlation in OBS_trend_pattern_correlation_10yr:
        file.write(f"{correlation}\n")

    file.write('\n30-year Trend Pattern Correlations:\n')
    for correlation in OBS_trend_pattern_correlation_30yr:
        file.write(f"{correlation}\n")

    file.write('\n60-year Trend Pattern Correlations:\n')
    for correlation in OBS_trend_pattern_correlation_60yr:
        file.write(f"{correlation}\n")

In [ ]:
OBS_trend_pattern_correlation_10yr

In [ ]:
OBS_unforced_trend_pattern_correlation_10yr = []
for i in range(len(trend_annual_da['10yr'].run)):
    OBS_unforced_trend_pattern_correlation_10yr.append(stats.pearsonr(MPI_ESM_unforced_10yr_trend.isel(run=i).values.flatten(), Obs_MPI_ESM_unforced_10yr_trend['tas'].values.flatten())[0])
    
OBS_unforced_trend_pattern_correlation_30yr = []
for i in range(len(trend_annual_da['30yr'].run)):
    OBS_unforced_trend_pattern_correlation_30yr.append(stats.pearsonr(MPI_ESM_unforced_30yr_trend.isel(run=i).values.flatten(), Obs_MPI_ESM_unforced_30yr_trend['tas'].values.flatten())[0])
    
OBS_unforced_trend_pattern_correlation_60yr = []
for i in range(len(trend_annual_da['60yr'].run)):
    OBS_unforced_trend_pattern_correlation_60yr.append(stats.pearsonr(MPI_ESM_unforced_60yr_trend.isel(run=i).values.flatten(), Obs_MPI_ESM_unforced_60yr_trend['tas'].values.flatten())[0])

In [ ]:
OBS_unforced_trend_pattern_correlation_30yr

In [ ]:
# Save to a text file
with open('obsUnforced_MPI_ESMunforced_pattern_correlations.txt', 'w') as file:
    file.write('10-year Trend Pattern Correlations:\n')
    for correlation in OBS_unforced_trend_pattern_correlation_10yr:
        file.write(f"{correlation}\n")

    file.write('\n30-year Trend Pattern Correlations:\n')
    for correlation in OBS_unforced_trend_pattern_correlation_30yr:
        file.write(f"{correlation}\n")

    file.write('\n60-year Trend Pattern Correlations:\n')
    for correlation in OBS_unforced_trend_pattern_correlation_60yr:
        file.write(f"{correlation}\n")

In [ ]:
client.close()
scluster.close()